In [ ]:
### install library
import pandas as pd
import requests
import sys
import os
import datetime
from bs4 import BeautifulSoup
from urllib.request import urlopen
import io
import re
#from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
#from pdfminer.pdfpage import PDFPage
#from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
#from pdfminer.layout import LAParams
#import multiprocessing

### functions
# progress bar
def progressBar(value, endvalue, bar_length=20):
    percent = float(value) / endvalue
    arrow = '-' * int(round(percent * bar_length) - 1) + '>'
    spaces = ' ' * (bar_length - len(arrow))

    sys.stdout.write("\rPercent: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()

In [ ]:
### get 산업분석리포트 from finance.naver.com
def naver_crawler_ind(ind, start_date='2020-01-03',end_date='2023-11-24'): ###
    print('\n.....crawling.....')
    start = datetime.datetime.now()
    url = 'https://finance.naver.com/research/industry_list.naver?'+'&page=' ###
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36',
    }
    idx_list = [2,3,4,5,6,10,11,12,13,14,18,19,20,21,22,26,27,28,29,30,34,35,36,37,38,42,43,44,45,46]
    df = pd.DataFrame(columns = ['industry','date','kapital','title','pdf'])

    ## 총 1049페이지까지 있음
    end_page=1049 #####1049
    for page in range(1, end_page+1):
        progressBar(page, end_page)
        end_flag = False
        pg_url = url+str(page)
        # get url source
        r = requests.get(pg_url, stream=True, headers=headers)
        r.encoding = 'euc-kr'
        html = r.text
        source = BeautifulSoup(html, 'html.parser')
        srlists = source.find_all('tr')

        for i in range(30):
            #tr = srlists[idx_list[i]]
            try:
                tr = srlists[idx_list[i]]
                # get data
                if tr.find_all('td')[0].text == ind:
                    date = '20'+tr.find_all('td')[4].text.replace('.','')
                    if idx_list[i] == len(srlists)-1:
                        end_flag = True
                        break
                    kap = tr.find_all('td')[2].text
                    pdf = tr.find_all('td')[3].find('a')['href'][-17:-4]
                    href = 'https://finance.naver.com/research/'+tr.find_all('td')[1].find('a')['href']
                    # get data from second url
                    html2 = urlopen(href)
                    sour2 = BeautifulSoup(html2.read(), "html.parser")

                    title = sour2.find_all("th")[0]
                    a1 = str(title).find('span')
                    a2 = len(str(title.find('span')))
                    b1 = str(title).find(str(title.find('p')))
                    title = str(title)[a1+a2:b1].replace('\n','')
                    title = title.replace('\t','')

                    #### //*[@id="contentarea_left"]/div[2]/table/tbody/tr[4]/td/div[1]
                    # XPath로 요소 선택
                    target_div = sour2.find('div', {'id': 'contentarea_left'}).find_all('div')[1]

                    # 해당 div 안에 있는 모든 tr 태그 선택
                    all_tr_tags = target_div.find_all('tr')[3]
                    # 각 p 태그의 텍스트를 리스트에 추가
                    all_text_list = [tr_tag.get_text() for tr_tag in all_tr_tags]

                    # 리스트의 모든 텍스트를 하나의 문자열로 이어붙이기
                    content = ' '.join(all_text_list)
                    record  = pd.Series([ind, date, kap, title, pdf, content],
                                     index = ['industry','date','kapital','title','pdf','content'])
                    df = df.append(record, ignore_index = True)
            except:
                pass



        if end_flag:
            break

    end = datetime.datetime.now()
    print('\n---- crawling TIME: '+str(end-start))

    return df

In [ ]:
df = naver_crawler_ind(ind='통신', start_date='2020-01-03',end_date='2023-11-24')


.....crawling.....
Percent: [>                   ] 2%

<ipython-input-6-70558bbd0ddd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)


Percent: [>                   ] 4%

<ipython-input-6-70558bbd0ddd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)
<ipython-input-6-70558bbd0ddd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)


Percent: [>                   ] 5%

...

<ipython-input-6-70558bbd0ddd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)


Percent: [------------------->] 100%

<ipython-input-6-70558bbd0ddd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)
<ipython-input-6-70558bbd0ddd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)



---- crawling TIME: 0:03:16.665326


<ipython-input-6-70558bbd0ddd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)


In [ ]:
df.to_csv('통신리포트_test.csv')

## 본격 크롤링 시작

In [ ]:
### install library
import pandas as pd
import requests
import sys
import os
import datetime
from bs4 import BeautifulSoup
from urllib.request import urlopen
import io
import re
#from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
#from pdfminer.pdfpage import PDFPage
#from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
#from pdfminer.layout import LAParams
#import multiprocessing

### functions
# progress bar
def progressBar(value, endvalue, bar_length=20):
    percent = float(value) / endvalue
    arrow = '-' * int(round(percent * bar_length) - 1) + '>'
    spaces = ' ' * (bar_length - len(arrow))

    sys.stdout.write("\rPercent: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()

In [ ]:
### get 산업분석리포트 from finance.naver.com
def naver_crawler_ind(ind, start_date='2007-11-12',end_date='2023-11-24'): ###
    print('\n.....crawling.....')
    start = datetime.datetime.now()
    url = 'https://finance.naver.com/research/industry_list.naver?'+'&page=' ###
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36',
    }
    idx_list = [2,3,4,5,6,10,11,12,13,14,18,19,20,21,22,26,27,28,29,30,34,35,36,37,38,42,43,44,45,46]
    df = pd.DataFrame(columns = ['industry','date','kapital','title','pdf'])

    ## 총 1049페이지까지 있음
    end_page=1049 #####1049
    for page in range(1, end_page+1):
        progressBar(page, end_page)
        end_flag = False
        pg_url = url+str(page)
        # get url source
        r = requests.get(pg_url, stream=True, headers=headers)
        r.encoding = 'euc-kr'
        html = r.text
        source = BeautifulSoup(html, 'html.parser')
        srlists = source.find_all('tr')

        for i in range(30):
            #tr = srlists[idx_list[i]]
            try:
                tr = srlists[idx_list[i]]
                # get data
                if tr.find_all('td')[0].text == ind:
                    date = '20'+tr.find_all('td')[4].text.replace('.','')
                    if idx_list[i] == len(srlists)-1:
                        end_flag = True
                        break
                    kap = tr.find_all('td')[2].text
                    pdf = tr.find_all('td')[3].find('a')['href'][-17:-4]
                    href = 'https://finance.naver.com/research/'+tr.find_all('td')[1].find('a')['href']
                    # get data from second url
                    html2 = urlopen(href)
                    sour2 = BeautifulSoup(html2.read(), "html.parser")

                    title = sour2.find_all("th")[0]
                    a1 = str(title).find('span')
                    a2 = len(str(title.find('span')))
                    b1 = str(title).find(str(title.find('p')))
                    title = str(title)[a1+a2:b1].replace('\n','')
                    title = title.replace('\t','')

                    #### //*[@id="contentarea_left"]/div[2]/table/tbody/tr[4]/td/div[1]
                    # XPath로 요소 선택
                    target_div = sour2.find('div', {'id': 'contentarea_left'}).find_all('div')[1]

                    # 해당 div 안에 있는 모든 tr 태그 선택
                    all_tr_tags = target_div.find_all('tr')[3]
                    # 각 p 태그의 텍스트를 리스트에 추가
                    all_text_list = [tr_tag.get_text() for tr_tag in all_tr_tags]

                    # 리스트의 모든 텍스트를 하나의 문자열로 이어붙이기
                    content = ' '.join(all_text_list)
                    record  = pd.Series([ind, date, kap, title, pdf, content],
                                     index = ['industry','date','kapital','title','pdf','content'])
                    df = df.append(record, ignore_index = True)
            except:
                pass



        if end_flag:
            break

    end = datetime.datetime.now()
    print('\n---- crawling TIME: '+str(end-start))

    return df

In [ ]:
df = naver_crawler_ind(ind='금융', start_date='2007-11-12',end_date='2023-11-24')


.....crawling.....
Percent: [>                   ] 1%

<ipython-input-13-69ae360d3caa>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)


Percent: [>                   ] 2%

<ipython-input-13-69ae360d3caa>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)


Percent: [>                   ] 2%

...

Percent: [------------------->] 98%

<ipython-input-13-69ae360d3caa>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)


Percent: [------------------->] 99%

<ipython-input-13-69ae360d3caa>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)


Percent: [------------------->] 100%
---- crawling TIME: 0:16:46.146099


In [ ]:
df.to_csv('금융_report_crawling.csv')

In [ ]:
for ind in ['에너지','해운','지주회사','디스플레이','화장품','자동차부품','교육','기타']:
  df = naver_crawler_ind(ind= ind,start_date='2007-11-12',end_date='2023-11-24')
  df.to_csv(f'{ind}_report_crawling.csv')


.....crawling.....
Percent: [>                   ] 0%

<ipython-input-4-f2f84b6ad382>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)


Percent: [>                   ] 0%

<ipython-input-4-f2f84b6ad382>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)


Percent: [>                   ] 1%

...

Percent: [------------------->] 100%

<ipython-input-4-f2f84b6ad382>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)
<ipython-input-4-f2f84b6ad382>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)
<ipython-input-4-f2f84b6ad382>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)


Percent: [------------------->] 100%

<ipython-input-4-f2f84b6ad382>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index = True)



---- crawling TIME: 0:59:32.574153
